In [1]:
import uuid
import pickle
import shutil
import os

from tldist.fingerprint.processing import FingerprintCalculatorResnet
from tldist.fingerprint.processing import calculate as fingerprint_calculate
from tldist.similarity import calculate as similarity_calculate
from tldist.data import Data
from tldist.database import get_database

In [2]:
fresnet = FingerprintCalculatorResnet()
fc_save = fresnet.save()

/Users/crjones/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/crjones/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
# Load the data
processing_dict = pickle.load(open('../data/hubble_acs.pck', 'rb'))

In [4]:
if os.path.isdir('basic_notebook.db'):
    shutil.rmtree('basic_notebook.db')
db = get_database('blitzdb', 'basic_notebook.db')

In [5]:
data = []
for fileinfo in processing_dict[:100]:
    im = Data(location=fileinfo['location'], radec=fileinfo['radec'], meta=fileinfo['meta'])
    data.append(im)
    db.save('data', im.save())

In [6]:
fingerprints = fingerprint_calculate(data, fc_save)
print([str(x) for x in fingerprints])
[db.save('fingerprint', x.save()) for x in fingerprints]

INFO  : Fingerprint 2018-05-04 13:34:03,343  
INFO  : Fingerprint 2018-05-04 13:34:04,065  Calculate 1000 predictions took 0.5956799983978271s
INFO  : Fingerprint 2018-05-04 13:34:04,435  Calculate 1000 predictions took 0.2803468704223633s
INFO  : Fingerprint 2018-05-04 13:34:04,853  Calculate 1000 predictions took 0.27882814407348633s
INFO  : Fingerprint 2018-05-04 13:34:05,235  Calculate 1000 predictions took 0.2834770679473877s
INFO  : Fingerprint 2018-05-04 13:34:05,621  Calculate 1000 predictions took 0.2939326763153076s
INFO  : Fingerprint 2018-05-04 13:34:05,990  Calculate 1000 predictions took 0.27366185188293457s
INFO  : Fingerprint 2018-05-04 13:34:06,380  Calculate 1000 predictions took 0.2987797260284424s
INFO  : Fingerprint 2018-05-04 13:34:06,750  Calculate 1000 predictions took 0.2724752426147461s
INFO  : Fingerprint 2018-05-04 13:34:07,140  Calculate 1000 predictions took 0.2977149486541748s
INFO  : Fingerprint 2018-05-04 13:34:07,624  Calculate 1000 predictions took 0.

INFO  : Fingerprint 2018-05-04 13:34:37,076  Calculate 1000 predictions took 0.2773091793060303s
INFO  : Fingerprint 2018-05-04 13:34:37,412  Calculate 1000 predictions took 0.2674689292907715s
INFO  : Fingerprint 2018-05-04 13:34:37,751  Calculate 1000 predictions took 0.2782599925994873s
INFO  : Fingerprint 2018-05-04 13:34:38,098  Calculate 1000 predictions took 0.2860100269317627s
INFO  : Fingerprint 2018-05-04 13:34:38,441  Calculate 1000 predictions took 0.28151988983154297s
INFO  : Fingerprint 2018-05-04 13:34:38,784  Calculate 1000 predictions took 0.2793691158294678s
INFO  : Fingerprint 2018-05-04 13:34:39,131  Calculate 1000 predictions took 0.2834310531616211s
INFO  : Fingerprint 2018-05-04 13:34:39,494  Calculate 1000 predictions took 0.3018617630004883s
INFO  : Fingerprint 2018-05-04 13:34:39,854  Calculate 1000 predictions took 0.2961580753326416s
INFO  : Fingerprint 2018-05-04 13:34:40,271  Calculate 1000 predictions took 0.3529238700866699s
INFO  : Fingerprint 2018-05-0

["Fingerprint ee1a18da-d2da-416d-acca-2c63c3911529 based on data 2910b60c-8da0-4225-a7fe-c6373ad68d18 with predictions [('n04286575', 'spotlight', 0.39021581411361694), ('n09229709', 'bubble', 0.1927020102739334), ('n02788148', 'bannister', 0.06545353680849075)]", "Fingerprint 6e8cb6d4-b56a-4abd-a6e6-9eb0ea061390 based on data 384a5294-6bcd-489f-923e-284eb0acd986 with predictions [('n03388043', 'fountain', 0.20394690334796906), ('n04589890', 'window_screen', 0.1391926258802414), ('n09229709', 'bubble', 0.08156129717826843)]", "Fingerprint 19ddf17a-fcb3-42f7-86f9-f3764e9cafe8 based on data b0ab5583-d4e9-42e6-aa60-6035f2b6f8aa with predictions [('n04404412', 'television', 0.47231248021125793), ('n03782006', 'monitor', 0.07505097985267639), ('n04152593', 'screen', 0.06339499354362488)]", "Fingerprint f616f17a-e01b-4155-a040-bd6a99cdccc3 based on data ea7b7f68-e8ec-43e5-b54d-9070a46f80d9 with predictions [('n02788148', 'bannister', 0.2345932573080063), ('n04286575', 'spotlight', 0.07664637

['ee1a18da-d2da-416d-acca-2c63c3911529',
 '6e8cb6d4-b56a-4abd-a6e6-9eb0ea061390',
 '19ddf17a-fcb3-42f7-86f9-f3764e9cafe8',
 'f616f17a-e01b-4155-a040-bd6a99cdccc3',
 'fb02c8ad-0091-4f6d-abb6-79365d458e10',
 'f851ec58-4a45-4471-9aea-16e8c7b2bd7c',
 '8dd5dd9e-e702-4699-b09d-2915b6c55730',
 '94312e32-447d-43b0-90b8-129936e0c1e6',
 '831e86f9-09b9-4ce5-b90a-3c8bd285f1e8',
 '99f5a166-293d-479c-85e5-88465fd4bf05',
 '4c562ab0-53d5-4b22-b93f-d8b9b57206c9',
 '1ae391e1-e0fc-4c14-bec6-b36381c6200f',
 '0023bcc5-5560-47bd-a0c1-683eaaef3721',
 'a5379934-7757-4bd1-85a8-0adacc657fbe',
 '2fd1328f-4714-4c00-880b-13de66bfe28a',
 '62360d6e-219f-4f0c-9a40-9669060705a7',
 '7c078736-bc16-4a47-9664-f3361b792cb9',
 '6fe39fdd-1875-490a-ace9-df704031af41',
 'd111835a-e7d1-4472-8398-81d15c8984c7',
 '1130b6d6-5345-43db-8524-753d56fde7e9',
 '527153f7-ce04-4763-915b-3b1646a0af4b',
 '87052d4c-96f7-4696-a2db-c43aa7710d20',
 'a196a8a1-134e-4352-86a2-5608d1996268',
 '5f42fe12-806a-4523-925d-a8bd0507f14c',
 '511fb690-4803-

In [7]:
similarity_tsne = similarity_calculate(fingerprints, 'tsne')
print(similarity_tsne)
db.save('similarity', similarity_tsne.save())

INFO  : Similarity 2018-05-04 13:34:44,246  Start threaded real_calculate 100 fingerprints and simcalc tsne
INFO  : Similarity 2018-05-04 13:34:44,247  Created tsne
INFO  : Similarity 2018-05-04 13:34:44,248  Going to calculate tSNE from 100 fingerprints
DEBUG : Similarity 2018-05-04 13:34:44,249      fingerprint is Fingerprint ee1a18da-d2da-416d-acca-2c63c3911529 based on data 2910b60c-8da0-4225-a7fe-c6373ad68d18 with predictions [('n04286575', 'spotlight', 0.39021581411361694), ('n09229709', 'bubble', 0.1927020102739334), ('n02788148', 'bannister', 0.06545353680849075)]
DEBUG : Similarity 2018-05-04 13:34:44,249      fingerprint is Fingerprint 6e8cb6d4-b56a-4abd-a6e6-9eb0ea061390 based on data 384a5294-6bcd-489f-923e-284eb0acd986 with predictions [('n03388043', 'fountain', 0.20394690334796906), ('n04589890', 'window_screen', 0.1391926258802414), ('n09229709', 'bubble', 0.08156129717826843)]
DEBUG : Similarity 2018-05-04 13:34:44,257      fingerprint is Fingerprint 19ddf17a-fcb3-42f7-

DEBUG : Similarity 2018-05-04 13:34:44,433      fingerprint is Fingerprint 511fb690-4803-4a51-a6b4-68af3d46558c based on data 9ce8f4b7-a96c-43ab-9c5d-33e58c4b873d with predictions [('n04589890', 'window_screen', 0.4482603669166565), ('n03196217', 'digital_clock', 0.1211838573217392), ('n03857828', 'oscilloscope', 0.08135665953159332)]
DEBUG : Similarity 2018-05-04 13:34:44,441      fingerprint is Fingerprint 5fcaf78b-5b06-4f33-b37f-57693154f452 based on data 5b9d7783-0082-489e-91a4-affeb9233eb2 with predictions [('n04589890', 'window_screen', 0.28881409764289856), ('n03388043', 'fountain', 0.06706307083368301), ('n03196217', 'digital_clock', 0.06351038813591003)]
DEBUG : Similarity 2018-05-04 13:34:44,450      fingerprint is Fingerprint 3f64c7ce-4a91-4b6e-94f3-a8a3a054e5df based on data 2ba26eb3-dd5c-4a1f-8ba3-93bebfb68cc0 with predictions [('n04589890', 'window_screen', 0.19562916457653046), ('n03196217', 'digital_clock', 0.14353200793266296), ('n03388043', 'fountain', 0.0883758217096

DEBUG : Similarity 2018-05-04 13:34:44,634      fingerprint is Fingerprint 9da53db0-df26-4517-a79c-5d254f858cdf based on data 63680cc1-1aed-4f7d-b570-a6b156ce128b with predictions [('n03028079', 'church', 0.08452477306127548), ('n09229709', 'bubble', 0.08418424427509308), ('n04418357', 'theater_curtain', 0.08182060718536377)]
DEBUG : Similarity 2018-05-04 13:34:44,642      fingerprint is Fingerprint e2d0e005-4ac7-4448-9ba4-650087e333f4 based on data 8e11e088-fd04-47e8-bebb-b12b73dfad88 with predictions [('n09229709', 'bubble', 0.10588624328374863), ('n04589890', 'window_screen', 0.09547604620456696), ('n03028079', 'church', 0.08961228281259537)]
DEBUG : Similarity 2018-05-04 13:34:44,649      fingerprint is Fingerprint 8b6924d1-2004-45d2-8e95-2de740b0105b based on data 27a6de54-1bcd-436c-a8ae-ac916d180d34 with predictions [('n04589890', 'window_screen', 0.13557378947734833), ('n04418357', 'theater_curtain', 0.09890268743038177), ('n02667093', 'abaya', 0.09479846805334091)]
DEBUG : Simi

DEBUG : Similarity 2018-05-04 13:34:44,864      fingerprint is Fingerprint 04f88bb9-5136-4866-b5bd-892df60b2ff4 based on data d2fae5ba-e34b-4e99-9e29-214c3b6d0e0a with predictions [('n04418357', 'theater_curtain', 0.16574500501155853), ('n02667093', 'abaya', 0.10534489154815674), ('n04589890', 'window_screen', 0.07597080618143082)]
DEBUG : Similarity 2018-05-04 13:34:44,877      fingerprint is Fingerprint 246a12d9-c726-4af4-a742-6eade7dae80e based on data 79e57f03-b449-4bcf-9766-242a51ffad04 with predictions [('n03388043', 'fountain', 0.09296679496765137), ('n02999410', 'chain', 0.07921164482831955), ('n04589890', 'window_screen', 0.0569804348051548)]
DEBUG : Similarity 2018-05-04 13:34:44,889      fingerprint is Fingerprint 7c8d3d4e-df68-41f6-b6b6-1a22d9d29147 based on data 41ca3fd8-d295-4078-9e40-4ee0b1d5fa3b with predictions [('n04418357', 'theater_curtain', 0.16658815741539001), ('n04589890', 'window_screen', 0.1070883721113205), ('n02667093', 'abaya', 0.07156546413898468)]
DEBUG :

DEBUG : Similarity 2018-05-04 13:34:46,027  X is [[3.90215814e-01 1.92702010e-01 6.54535368e-02 ... 9.97072931e-08
  9.08432440e-08 6.86983412e-08]
 [4.98547545e-03 8.15612972e-02 1.98413036e-03 ... 1.11310975e-07
  1.57535865e-07 9.52887973e-08]
 [9.79334302e-03 1.14302023e-03 4.23568347e-03 ... 2.30153518e-07
  4.22535464e-07 2.16757570e-07]
 ...
 [1.84778363e-01 6.55464530e-02 7.92061165e-02 ... 5.43855094e-07
  3.68071824e-07 4.16490195e-07]
 [1.01338707e-01 3.69550139e-02 9.70217884e-02 ... 9.02292072e-07
  4.01680438e-07 7.67629103e-07]
 [1.88028574e-01 6.55660182e-02 5.87620102e-02 ... 5.36354491e-07
  3.95399638e-07 3.58108508e-07]]
DEBUG : Similarity 2018-05-04 13:34:46,028  Fingerprint list [<tldist.fingerprint.fingerprint.Fingerprint object at 0x1237117f0>, <tldist.fingerprint.fingerprint.Fingerprint object at 0x12964c3c8>, <tldist.fingerprint.fingerprint.Fingerprint object at 0x12ad87eb8>, <tldist.fingerprint.fingerprint.Fingerprint object at 0x129461fd0>, <tldist.fingerpri

INFO  : Similarity 2018-05-04 13:34:47,359  Done calculation
INFO  : Similarity 2018-05-04 13:34:47,360  Returning the dictionary of information


Similarity tsne based on []...


'c7e10b09-2e55-4aeb-a936-fefd61718328'

In [8]:
len(db.find('fingerprint'))

100